In [ ]:
import telebot
from config import TELEGRAM_TOKEN  # Добавьте ваш токен в config.py

# Ваш существующий код
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd 
import pymorphy3 as pm

morph = pm.MorphAnalyzer()
normalized = pd.read_csv('normalized.csv')

vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 5))
tfidf_matrix = vectorizer.fit_transform(normalized['overview'])

# Создаем бота
bot = telebot.TeleBot(TELEGRAM_TOKEN)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Привет! Напиши описание фильма, который хочешь найти, и я подберу похожие.")

@bot.message_handler(func=lambda message: True)
def find_similar_movies(message):
    try:
        user_query = message.text
        s = ''

        STOP_WORDS = {'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она',
    'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее',
    'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда',
    'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до',
    'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей',
    'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем',
    'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет',
    'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним',
    'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда',
    'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть',
    'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая',
    'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед',
    'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно',
    'всю', 'между', 'фильм'}
        
        # Нормализация запроса
        for i in user_query.split():
            if i.lower() not in STOP_WORDS:  # Удаляем стоп-слова
                s += morph.parse(i)[0].normal_form + ' '

        # Поиск похожих фильмов
        query_vector = vectorizer.transform([s])
        cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)
        similarity_scores = cosine_similarities[0]

        non_zero_indices = np.where(similarity_scores > 0.1)[0]
        non_zero_scores = similarity_scores[non_zero_indices]

        if len(non_zero_indices) > 0:
            sorted_indices = non_zero_indices[np.argsort(non_zero_scores)[::-1]]
            top_indices = sorted_indices[:5]
            top_scores = similarity_scores[top_indices]

            response = f"🎬 Запрос: '{user_query}'\n\n"
            response += f"Наиболее подходящие фильмы (найдено {len(top_indices)}):\n\n"

            for i, (idx, score) in enumerate(zip(top_indices, top_scores), 1):
                best_match_title = normalized.loc[idx, 'movie']
                response += f"{i}. '{best_match_title}' (сходство: {score:.4f})\n"

        else:
            response = "❌ Не найдено подходящих фильмов с ненулевым сходством."

        bot.reply_to(message, response)

    except Exception as e:
        bot.reply_to(message, f"Произошла ошибка: {str(e)}")

# Запуск бота
if __name__ == "__main__":
    print("Бот запущен...")
    bot.polling()